In [1]:
import subprocess
import pandas as pd
import lavaburst
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import tadtool.tad as tad

In [2]:
# set initial properties for data location
res = 40000
data_location = "matrices/"
armatus_location = "./armatus"
out_location = "yielded/"
ins_score_data = data_location + "ins_score_input/"
noise_values = [4, 8, 12, 16, 20]
sim_values = list(range(1, 6))

In [8]:
armatus_gamma = [i / 10 for i in range(-5, 6)]
modularity_gamma = list(range(101))
potts_gamma = list(range(41))
variance_gamma = list(range(101))
corner_gamma = [1, 2]

In [9]:
# run armatus with all gamma values
for noise in noise_values:
    for sim in sim_values:
        for gamma in armatus_gamma:
            print(f"\rnoise={noise} sim={sim} gamma={gamma}", end="")
            subprocess.run(f"bash -c '{armatus_location} -i {data_location}simHiC_countMatrix_noise{noise}_sim{sim}.txt.gz -g {gamma} -j -o {out_location}armatus_gamma{gamma}_noise{noise}_sim{sim} -r {res}'")
print("\rFinished.")

Finished.sim=5 gamma=0.51


In [4]:
# convert armatus tad coordinates for further handling
custom_range = [i / 2 for i in range(13)]
for gamma in armatus_gamma:
    tads = pd.read_csv(f"{out_location}armatus_gamma{gamma}_noise{noise}_sim{sim}.txt", comment = "#", sep = "\t", header=None)
    del tads[0]
    tads[2] = tads[2] + 1
    tads.to_csv(f"{out_location}armatus_gamma{gamma}_noise{noise}_sim{sim}.txt", sep="\t", header=None, index=False)
    

In [4]:
#load matrices for lavaburst
matrices = pd.DataFrame(index=noise_values, columns=sim_values)
good_bins = pd.DataFrame(index=noise_values, columns=sim_values)
for noise in noise_values:
    for sim in sim_values:
        print(f"\r noise={noise} sim={sim}", end="")
        matrices.loc[noise, sim] = np.loadtxt(f"{data_location}simHiC_countMatrix_noise{noise}_sim{sim}.txt")
        good_bins.loc[noise, sim] = matrices.loc[noise, sim].astype(bool).sum(axis=0) > 100
print("\r finished")

 finished sim=5


In [15]:
# run lavaburst with modularity score
for noise in noise_values:
    for sim in sim_values:
        for gamma in modularity_gamma:
            print(f"\r noise={noise} sim={sim} gamma={gamma} in progress", end="")
            S = lavaburst.scoring.modularity_score(matrices.loc[noise, sim], gamma=gamma, binmask=good_bins.loc[noise, sim])
            model = lavaburst.SegModel(S)
            segments = model.optimal_segmentation()
            np.savetxt(f"{out_location}lava_modularity_noise{noise}_sim{sim}_gamma{gamma}.txt", segments * res, delimiter="\t", fmt="%d")

 noise=20 sim=5 gamma=100 in progress

In [27]:
#run lavaburst with armatus
for noise in noise_values:
    for sim in sim_values:
        for gamma in armatus_gamma:
            print(f"\rnoise={noise} sim={sim} gamma={gamma} in progress", end="")
            S = lavaburst.scoring.armatus_score(matrices.loc[noise, sim], gamma=gamma, binmask=good_bins.loc[noise, sim])
            model = lavaburst.SegModel(S)
            segments = model.optimal_segmentation()
            np.savetxt(f"{out_location}lava_armatus_noise{noise}_sim{sim}_gamma{gamma}.txt", segments * res, delimiter="\t", fmt="%d")
print("\rFinished.")

Finished.sim=5 gamma=0.5 in progresss


In [14]:
#run lavaburst with potts score
for noise in noise_values:
    for sim in sim_values:
        for gamma in potts_gamma:
            print(f"\rnoise={noise} sim={sim} gamma={gamma} in progress", end="")
            S = lavaburst.scoring.potts_score(matrices.loc[noise, sim], gamma=gamma, binmask=good_bins.loc[noise, sim])
            model = lavaburst.SegModel(S)
            segments = model.optimal_segmentation()
            np.savetxt(f"{out_location}lava_potts_noise{noise}_sim{sim}_gamma{gamma}.txt", segments * res, delimiter="\t", fmt="%d")
print("\rFinished.")

Finished.sim=5 gamma=40 in progress


In [ ]:
# run lavaburst with variance score
for noise in noise_values:
    for sim in sim_values:
        for gamma in variance_gamma:
            print(f"\r noise={noise} sim={sim} gamma={gamma} in progress", end="")
            S = lavaburst.scoring.variance_score(matrices.loc[noise, sim], gamma=gamma, binmask=good_bins.loc[noise, sim])
            model = lavaburst.SegModel(S)
            segments = model.optimal_segmentation()
            np.savetxt(f"{out_location}lava_variance_noise{noise}_sim{sim}_gamma{gamma}.txt", segments * res, delimiter="\t", fmt="%d")

In [7]:
# run lavaburst with corner score
for noise in noise_values:
    for sim in sim_values:
        for gamma in corner_gamma:
            print(f"\r noise={noise} sim={sim} gamma={gamma} in progress", end="")
            S = lavaburst.scoring.corner_score(matrices.loc[noise, sim], gamma=gamma, binmask=good_bins.loc[noise, sim])
            model = lavaburst.SegModel(S)
            segments = model.optimal_segmentation()
            np.savetxt(f"{out_location}lava_corner_noise{noise}_sim{sim}_gamma{gamma}.txt", segments * res, delimiter="\t", fmt="%d")

 noise=20 sim=5 gamma=2 in progress

In [11]:
# convert matrices for insulation score DEPRECATED
for noise in noise_values:
    for sim in sim_values:
        ndim = matrices.loc[noise, sim].shape[0]
        bin_names = [f"bin{bin}|sim{sim}noise{noise}|chr1:{bin * res}-{bin * res +res}" for bin in range(ndim)]
        ins_score_matrix = np.empty(shape=(ndim + 1, ndim + 1), dtype="object")
        ins_score_matrix[1:, 1:] = matrices.loc[noise, sim]
        ins_score_matrix[0, 1:] = bin_names
        ins_score_matrix[1:, 0] = bin_names
        ins_score_matrix[0,0] = ""
        np.savetxt(f"{ins_score_data}simHiC_countMatrix_noise{noise}_sim{sim}.txt", ins_score_matrix, delimiter="\t", fmt="%s")

In [10]:
# make region bed files for tadtool
for noise in noise_values:
    for sim in sim_values:
        ndim = matrices.loc[noise, sim].shape[0]
        regions = np.empty(shape=(ndim, 3), dtype="object")
        regions[:, 0] = "chr1"
        starts = np.array([i * res for i in range(ndim)], dtype="int")
        regions[:, 1] = starts
        regions[:, 2] = starts + res
        np.savetxt(f"{data_location}region_noise{noise}_sim{sim}.txt", regions, delimiter="\t", fmt="%s")

In [7]:
# load region files for tadtool
regions = pd.DataFrame(index=noise_values, columns=sim_values)
for noise in noise_values:
    for sim in sim_values:
        print(f"\r noise={noise} sim={sim}", end="")
        regions.loc[noise, sim], some_ix = tad.load_regions(f"{data_location}region_noise{noise}_sim{sim}.txt")

 noise=20 sim=5

In [28]:
# set ranges of values for tadtool
cutoff_values = list(range(11))
window_values = [res * i for i in list(np.arange(0.5, 5, 0.5))]

In [30]:
# run tadtool with insulation score
for cutoff in cutoff_values:
    for window in window_values:
        for noise in noise_values:
            for sim in sim_values:
                print(f"\r noise={noise} sim={sim} window={window} cutoff={cutoff}", end="")
                ii = tad.insulation_index(matrices.loc[noise, sim], regions.loc[noise, sim], window_size=window)
                tads = tad.call_tads_insulation_index(ii, cutoff, regions=regions.loc[noise, sim])
                with open(f"{out_location}ii_noise{noise}_sim{sim}_window{window}_cutoff{cutoff}.txt", "w") as outfile:
                    for some_tad in tads:
                        outfile.write(f"{some_tad.start - 1}\t{some_tad.end}\n")

 window=180000.0 cutoff=10

In [33]:
# run tadtool with directionality index
cutoff_values = list(range(5, 16))
window_values = [i * res for i in range(2, 5)]
for cutoff in cutoff_values:
    for window in window_values:
        for noise in noise_values:
            for sim in sim_values:
                print(f"\r sim={sim} noise={noise} window={window} cutoff={cutoff}", end="")
                di = tad.directionality_index(matrices.loc[noise, sim], regions.loc[noise, sim], window_size=window)
                tads = tad.call_tads_directionality_index(di, cutoff, regions=regions.loc[noise, sim])
                with open(f"{out_location}di_noise{noise}_sim{sim}_window{window}_cutoff{cutoff}.txt", "w") as outfile:
                    for some_tad in tads:
                        outfile.write(f"{some_tad.start - 1}\t{some_tad.end}\n")

 sim=5 noise=20 window=160000 cutoff=15